In [77]:
import os
import pickle
import platform
import itertools
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [10]:
if platform.system() == 'Windows':
    if platform.release() == '10':
        path = r'D:\CIS-PD Study\Subjects' #Windows remote path
        folder_path = r'D:\CIS-PD Study' #generic Windows repo path
        dict_path = 'D:\CIS-PD Study\Data_dict'
        scores_path = r'D:\CIS-PD Study\Scores' #remote repo
        features_path = r'D:\CIS-PD Study\FeatureMatrix' #remote repo
        
    elif platform.release() == '7':
        path = r'Y:\CIS-PD Study\Subjects'
        folder_path = r'Y:\CIS-PD Study'
#         dict_path = r'C:\Users\adai\Documents\Data_dict'
        dict_path = r'Y:\CIS-PD Study\Data_dict'
        scores_path = r'Y:\CIS-PD Study\Scores'
        features_path = r'Y:\CIS-PD Study\FeatureMatrix'
    
else:
    path = '/Volumes/RTO/CIS-PD Study/Subjects/' #Mac
    folder_path = '/Volumes/RTO/CIS-PD Study/'
    dict_path = '../Data_dict' # Mac local path
    scores_path = '../Scores/' # Mac local path
    features_path = '../FeatureMatrix' # Mac local path

In [92]:
namesuffix = 'HP+LP.pkl'
datafile = 'DataRaw_AllLocs'+namesuffix
featfile = 'Features_AllLocs'+namesuffix
output = 'MetaFeatures_'+namesuffix

Data = pickle.load(open(os.path.join(features_path,featfile),'rb'))
Data = Data.reset_index(drop=True)
print(Data.shape)
metadata = pickle.load(open(os.path.join(features_path,datafile),'rb'))
metadata=metadata.reset_index(drop=True)
metadata = metadata.iloc[:,:7]
print(metadata.shape)
#concatenates metadata
Data=pd.concat((metadata,Data),axis=1)
print(Data.shape)

(48579, 6)
(48579, 7)
(48579, 13)


In [93]:
Data.head()

,Subject,Visit,Side,Task,Tremor,Bradykinesia,Dyskinesia,anterior_thigh__accel,anterior_thigh__gyro,dorsal_hand__accel,dorsal_hand__gyro,sacrum_accel,sacrum_gyro
0,1004,2 Weeks: Time 0,left,Motor #2: Walking,0.0,1.0,0.0,"[0.014046611028026703, 0.020603451774853176, 0...","[10.273506428273729, 2.706283216342912, 5.0615...","[0.007989930813309471, 0.00961279888662689, 0....","[2.7635771865880097, 1.922690587454126, 4.2009...","[0.002870561203119263, 0.011362069633733492, 0...","[0.8215614656047298, 1.1265178529231297, 0.664..."
1,1004,2 Weeks: Time 0,left,Motor #2: Walking,0.0,1.0,0.0,"[0.012784975789800944, 0.019815286722809262, 0...","[10.66239472590443, 2.6287032265762083, 4.4656...","[0.007692838138012616, 0.009210303186171182, 0...","[3.3284323758047036, 2.822697515889742, 5.3517...","[0.003073760806722303, 0.010809689170103418, 0...","[0.7898793130704667, 3.3456375775178198, 0.722..."
2,1004,2 Weeks: Time 0,left,Motor #2: Walking,0.0,1.0,0.0,"[0.011073270092395962, 0.017390263992478575, 0...","[9.008067810874358, 2.1913443809962745, 4.1487...","[0.006767186243379367, 0.009203679619930478, 0...","[2.6885012576952496, 2.8744981875353393, 4.437...","[0.003317036338904409, 0.009088831731464318, 0...","[0.7058148993143553, 3.472527979175533, 0.7433..."
3,1004,2 Weeks: Time 0,left,Motor #2: Walking,0.0,1.0,0.0,"[0.014497770683490268, 0.020706346408661484, 0...","[9.19704928911273, 2.470238328817465, 5.191160...","[0.007352144848440469, 0.01013037529611546, 0....","[2.499042347499924, 1.2004465412231244, 4.3006...","[0.002986414736949026, 0.012085768375826016, 0...","[0.7541402229259171, 1.430995228538481, 0.7466..."
4,1004,2 Weeks: Time 0,left,Motor #2: Walking,0.0,1.0,0.0,"[0.013174749215279774, 0.021019617531214934, 0...","[9.574205482842476, 2.41849860452593, 5.094296...","[0.007084818856755019, 0.00880787596180788, 0....","[3.261874206390144, 2.9412816369694337, 4.6892...","[0.003197768967434297, 0.01121167717474376, 0....","[0.8274834438067099, 2.65230166410693, 0.86461..."


In [94]:
#filter empty rows on all locations
for s in Data.columns.values[7:]:
    print('%s,discarded %d rows'%(s,sum(Data[s].apply(type) == float)))
    Data = Data[Data[s].apply(type) != float]

anterior_thigh__accel,discarded 6731 rows
anterior_thigh__gyro,discarded 0 rows
dorsal_hand__accel,discarded 3 rows
dorsal_hand__gyro,discarded 0 rows
sacrum_accel,discarded 4 rows
sacrum_gyro,discarded 0 rows


In [95]:
#filter empty rows (no features available) on hand sensors
print('discarded %d rows'%(len(Data[Data.dorsal_hand__accel.apply(type) == float])))
Data = Data[Data.dorsal_hand__accel.apply(type) != float]
print('discarded %d rows'%(len(Data[Data.dorsal_hand__gyro.apply(type) == float])))
Data = Data[Data.dorsal_hand__gyro.apply(type) != float]

discarded 0 rows
discarded 0 rows


In [96]:
#unpack features
colnames=['RMSX', 'RMSY', 'RMSZ', 'rangeX', 'rangeY', 'rangeZ', 'meanX',
       'meanY', 'meanZ', 'varX', 'varY', 'varZ', 'skewX', 'skewY', 'skewZ',
       'kurtX', 'kurtY', 'kurtZ', 'xcor_peakXY', 'xcorr_peakXZ',
       'xcorr_peakYZ', 'xcorr_lagXY', 'xcorr_lagXZ', 'xcorr_lagYZ', 'Dom_freq',
       'Pdom_rel', 'PSD_mean', 'PSD_std', 'PSD_skew', 'PSD_kur', 'jerk_mean',
       'jerk_std', 'jerk_skew', 'jerk_kur', 'Sen_X', 'Sen_Y', 'Sen_Z']

sensor_list = ['dorsal_hand_']


colnames1=[i+'acc' for i in colnames]
colnames2=[i+'gyr' for i in colnames]
colnames=colnames1+colnames2

len(Data)

Datafinal = pd.DataFrame()

In [97]:
for i in range(len(Data)):
    Datatemp=pd.DataFrame()
    for sensors in sensor_list:
        F1 = Data[sensors + '_accel'].iloc[i]
        F2 = Data[sensors + '_gyro'].iloc[i]
        F = pd.DataFrame(data=np.hstack((F1,F2)).reshape(-1,1).T,index=[i],columns=[i for i in colnames])
        Datatemp=pd.concat((Datatemp,F),axis=1)
    Datafinal = pd.concat((Datafinal,Datatemp),axis=0)

Datafinal=Datafinal.reset_index(drop=True)
# Datanew=Datanew.reset_index(drop=True)
Data=Data.reset_index(drop=True)

Data = pd.concat((Data.iloc[:,:7],Datafinal),axis=1)

In [98]:
Data.head(2)

,Subject,Visit,Side,Task,Tremor,Bradykinesia,Dyskinesia,RMSXacc,RMSYacc,RMSZacc,...,PSD_stdgyr,PSD_skewgyr,PSD_kurgyr,jerk_meangyr,jerk_stdgyr,jerk_skewgyr,jerk_kurgyr,Sen_Xgyr,Sen_Ygyr,Sen_Zgyr
0,1004,2 Weeks: Time 0,left,Motor #2: Walking,0.0,1.0,0.0,0.007990,0.009613,0.006777,...,333.369231,3.587466,15.204802,0.114246,10.726893,0.176444,-0.009173,0.416625,0.571324,0.438326
1,1004,2 Weeks: Time 0,left,Motor #2: Walking,0.0,1.0,0.0,0.007693,0.009210,0.006636,...,511.580241,4.261274,17.609498,-0.173831,10.278264,0.551148,0.059904,0.321724,0.525199,0.307797


In [99]:
#drop features 
Data=Data.drop(labels=['RMSXacc','RMSYacc','RMSZacc'],axis=1)    #equivalent to variance if mean 0
Data=Data.drop(labels=['meanXacc','meanYacc','meanZacc'],axis=1) #if signal is mean 0 this feature is useless
Data=Data.drop(labels=['varXacc','varYacc','varZacc'],axis=1) #range is strongly correlated with variance
Data=Data.drop(labels=['RMSXgyr','RMSYgyr','RMSZgyr'],axis=1)    #equivalent to variance if mean 0
Data=Data.drop(labels=['meanXgyr','meanYgyr','meanZgyr'],axis=1) #if signal is mean 0 this feature is useless
Data=Data.drop(labels=['varXgyr','varYgyr','varZgyr'],axis=1) #range is strongly correlated with variance

In [100]:
Data = Data[Data.Subject!=1020]
DataO = Data.copy()

In [127]:
Data.Visit.unique()

array(['2 Weeks: Time 0', '2 Weeks: Time 30', '2 Weeks: Time 60',
       '2 Weeks: Time 90', '2 Weeks: Time 120', '2 Weeks: Time 150',
       '4 Weeks'], dtype=object)

In [128]:
# model = svm.SVC(probability=True)
model = RandomForestClassifier(n_estimators=50,random_state=2)
Data = DataO.copy()
Data = Data[Data.Subject!=1020]
Data = Data[~Data['Bradykinesia'].isnull()]
Data.loc[Data['Bradykinesia']>1,'Bradykinesia'] = 1
Dist = pd.DataFrame(columns = Data.Subject.unique(),index = Data.Subject.unique())
for s in itertools.permutations(Data.Subject.unique(),2):
    train = Data[Data.Subject==s[0]]
    test = Data[(Data.Subject==s[1])&((Data.Visit=='2 Weeks: Time 0')|(Data.Visit=='2 Weeks: Time 30'))]
    model.fit(train.iloc[:,7::],train.Bradykinesia)
    Dist.loc[s[0],s[1]] = roc_auc_score(test.Bradykinesia,model.predict_proba(test.iloc[:,7::])[:,1])

In [136]:
model = RandomForestClassifier(n_estimators=50,random_state=2)
auc_all = []
for s in Data.Subject.unique():
    try:
        SubDist = Dist.loc[:,s].sort_values(ascending=False)
        trainSubjs = SubDist[SubDist>.75].index.values
        if len(trainSubjs)==0:
            trainSubjs = SubDist.index[0:2].values
        trainInds = [any(sub==trainSubjs) for sub in Data.Subject]
        train = Data[trainInds]
        model.fit(train.iloc[:,7::],train.Bradykinesia)
        test = Data[(Data.Subject==s)&((Data.Visit!='2 Weeks: Time 0')&(Data.Visit!='2 Weeks: Time 30'))]
        test_score = model.predict_proba(test.iloc[:,7::])
        auroc = roc_auc_score(test.Bradykinesia,test_score[:,1])
        print(s, auroc)
        auc_all.append(auroc)
    except:
        continue

1004 0.5992154674369747
1016 0.7365994902381243
1018 0.8119973853869167
1019 0.9180287652798027
1024 0.7212791223228798
1029 0.6193887998183352
1030 0.7659437946494011
1032 0.7723714548680627
1038 0.6537640879746144
1044 0.8257066293438929
1046 0.7598463499731616
1049 0.4709399772148426
1051 0.5716099692889204
1052 0.7118146636872955
1053 0.9450484764542936
1054 0.5939825890483785
1055 0.7118711520620731
1056 0.8134461947626841


In [125]:
Dist

,1004,1016,1018,1019,1024,1029,1030,1032,1038,1044,1046,1047,1049,1051,1052,1053,1054,1055,1056
1004,NaN,0.979,0.647067,0.747917,0.961497,0.460401,0.663146,0.900128,0.692824,0.660844,0.673634,0.651923,0.671836,0.661207,0.733293,0.764955,0.965971,0.713773,0.480307
1016,0.845082,NaN,0.668126,0.499922,0.9875,0.908548,0.566901,0.906116,0.234241,0.751591,0.423565,0.610214,0.322994,0.512819,0.830303,0.628339,0.908475,0.821118,0.668704
1018,0.786141,0.830571,NaN,0.74474,0.713272,0.487746,0.743765,0.7855,0.337907,0.748182,0.343645,0.577991,0.461404,0.71085,0.882229,0.714621,0.917601,0.926172,0.680205
1019,0.64775,0.888286,0.627697,NaN,0.860494,0.338223,0.582013,0.869226,0.810678,0.788766,0.718297,0.809402,0.593805,0.795266,0.68774,0.784598,0.957627,0.7521,0.599058
1024,0.878335,0.897571,0.604674,0.640911,NaN,0.679041,0.68827,0.875642,0.354131,0.808019,0.508396,0.705598,0.388432,0.503184,0.817415,0.67505,0.926728,0.774164,0.743443
1029,0.826961,0.950286,0.509271,0.469167,0.942824,NaN,0.464129,0.923011,0.606204,0.663312,0.698997,0.662692,0.543169,0.675534,0.566501,0.675476,0.970274,0.375172,0.492233
1030,0.697995,0.878714,0.657407,0.76612,0.874306,0.41549,NaN,0.810308,0.525515,0.857435,0.730421,0.643376,0.659774,0.73163,0.737136,0.685635,0.902738,0.638723,0.541338
1032,0.943382,0.972429,0.619184,0.617188,0.924923,0.676506,0.687097,NaN,0.46037,0.685097,0.47091,0.515641,0.413596,0.460369,0.803637,0.629902,0.929335,0.808597,0.665054
1038,0.352316,0.510571,0.414007,0.641719,0.488426,0.384885,0.601819,0.665419,NaN,0.518636,0.702062,0.625726,0.665452,0.55907,0.352751,0.522734,0.421512,0.325164,0.282659
1044,0.74021,0.809,0.673847,0.767031,0.74892,0.482434,0.75022,0.778978,0.391652,NaN,0.625732,0.715128,0.572906,0.69514,0.627238,0.653843,0.845893,0.610322,0.619684


In [122]:
auc_all1